In [1]:
import os
import scanpy as sc
import anndata as ad
import glob
# import pyreadr

import h5py

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix


In [2]:
gs1_dir = "data/spotless/gold_standard_1/"

In [3]:
fpaths = sorted(glob.glob(os.path.join(gs1_dir, "*.h5ad")))
fovs = [sc.read_h5ad(name) for name in fpaths]
fovs

[AnnData object with n_obs × n_vars = 9 × 10000
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'spot_composition.Endothelial', 'spot_composition.Excitatory.layer.3', 'spot_composition.Excitatory.layer.4', 'spot_composition.Excitatory.layer.II', 'spot_composition.Interneurons', 'spot_composition.Microglia', 'spot_composition.Astrocytes.deep', 'spot_composition.Astrocytes.superficial', 'spot_composition.Interneurons.deep', 'spot_composition.OPC', 'spot_composition.spot_no', 'relative_spot_composition.Endothelial', 'relative_spot_composition.Excitatory.layer.3', 'relative_spot_composition.Excitatory.layer.4', 'relative_spot_composition.Excitatory.layer.II', 'relative_spot_composition.Interneurons', 'relative_spot_composition.Microglia', 'relative_spot_composition.Astrocytes.deep', 'relative_spot_composition.Astrocytes.superficial', 'relative_spot_composition.Interneurons.deep', 'relative_spot_composition.OPC', 'relative_spot_composition.spot_no', 'coordinates.x', 'coordinates.y'
  

In [4]:
obs_cols = sorted(list(set.union(*[set(fov.obs.columns) for fov in fovs])))

In [5]:
for fov in fovs:
    fov.obs = fov.obs.reindex(columns=obs_cols)

In [6]:
# for column in fovs[0].obs.columns:
#     display(fovs[0].obs[column])
[os.path.basename(f).split(".")[0] for f in fpaths]

['Eng2019_cortex_svz_fov0',
 'Eng2019_cortex_svz_fov1',
 'Eng2019_cortex_svz_fov2',
 'Eng2019_cortex_svz_fov3',
 'Eng2019_cortex_svz_fov4',
 'Eng2019_cortex_svz_fov5',
 'Eng2019_cortex_svz_fov6']

In [12]:
gs1_ref_path = "data/spotless/reference/gold_standard_1.h5ad"

In [13]:
gs1_ref = sc.read_h5ad(gs1_ref_path)

In [14]:
gs1_ref

AnnData object with n_obs × n_vars = 906 × 10000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'louvain', 'celltype', 'Field.of.View', 'X', 'Y', 'celltype_coarse', 'Region'
    var: 'features'

In [6]:
# adata.obs

In [11]:
# result = pyreadr.read_r('data/spotless/gold_standard_1/Eng2019_cortex_svz_fov0.rds')
with h5py.File("data/spotless/gold_standard_1.h5" , "r") as f:
    for name in f:
        gene_names = [str(gene, "utf-8") for gene in f[name]["geneNames"][()]]
        ad.AnnData(csr_matrix(f[name]["counts"][()]), dtype=np.int64)

AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
